<a href="https://colab.research.google.com/github/RushikuwarRK/my_train_model-image-generate-/blob/main/my_train_model(image_generate).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow   # or torch for PyTorch
!pip install numpy matplotlib

In [2]:
# Load and preprocess your dataset:
import tensorflow as tf

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalize images to [-1, 1] range
x_train = (x_train - 127.5) / 127.5
x_train = x_train.reshape(x_train.shape[0], 32, 32, 3)  # Ensure correct shape

170498071/170498071 [==============================] - 2s 0us/step


In [3]:
# Build the Generator and Discriminator models:
from tensorflow.keras.layers import Dense, Reshape, Flatten, BatchNormalization, LeakyReLU, Conv2DTranspose
from tensorflow.keras.models import Sequential

# Generator model
def build_generator():
    model = Sequential()
    model.add(Dense(256, input_dim=100))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(32 * 32 * 3, activation='tanh'))
    model.add(Reshape((32, 32, 3)))
    return model

# Discriminator model
def build_discriminator():
    model = Sequential()
    model.add(Flatten(input_shape=(32, 32, 3)))
    model.add(Dense(512))
    model.add(LeakyReLU(0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [4]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import numpy as np

# Ensure you have a proper discriminator and generator model.
# For illustration, we assume `build_discriminator()` and `build_generator()` are defined elsewhere.

# Using the legacy Adam optimizer
optimizer = tf.keras.optimizers.legacy.Adam(0.0002, 0.5)

# Build the discriminator model
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Build the generator model
generator = build_generator()

# GAN Setup (combined model for training the generator)
z = tf.keras.Input(shape=(100,))  # Random noise input to the generator
img = generator(z)  # Generated image
discriminator.trainable = False  # We don't want to train the discriminator when training the generator
valid = discriminator(img)  # Check if the generated image is valid (real or fake)

# Combined model that takes noise and produces output from the discriminator
combined = tf.keras.Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

# Training function
def train(epochs, batch_size=128):
    half_batch = int(batch_size / 2)

    for epoch in range(epochs):
        # Train the discriminator
        idx = np.random.randint(0, x_train.shape[0], half_batch)  # Random index for real images
        imgs = x_train[idx]

        noise = np.random.normal(0, 1, (half_batch, 100))  # Random noise for generating fake images
        gen_imgs = generator.predict(noise)  # Generate fake images

        # Train on real images (label them as real, i.e., 1)
        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))

        # Train on fake images (label them as fake, i.e., 0)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))

        # Combine the losses from real and fake images
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator (we want to make the discriminator believe the fake images are real)
        noise = np.random.normal(0, 1, (batch_size, 100))  # Generate random noise
        g_loss = combined.train_on_batch(noise, np.ones((batch_size, 1)))  # Target is real labels (1)

        # Print the progress
        print(f"{epoch} [D loss: {d_loss[0]}] [G loss: {g_loss}]")

# Start training
train(epochs=10000, batch_size=64)


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 25ms/step
7500 [D loss: 0.39881421625614166] [G loss: 2.478422164916992]
1/1 [==============================] - 0s 27ms/step
7501 [D loss: 0.49505938589572906] [G loss: 1.9920711517333984]
1/1 [==============================] - 0s 26ms/step
7502 [D loss: 0.5226041972637177] [G loss: 1.8809425830841064]
1/1 [==============================] - 0s 25ms/step
7503 [D loss: 0.45833277702331543] [G loss: 2.369122266769409]
1/1 [==============================] - 0s 26ms/step
7504 [D loss: 0.263838030397892] [G loss: 3.8006155490875244]
1/1 [==============================] - 0s 25ms/step
7505 [D loss: 0.31727419793605804] [G loss: 4.994504928588867]
1/1 [==============================] - 0s 29ms/step
7506 [D loss: 0.3765086233615875] [G loss: 2.3393189907073975]
1/1 [==============================] - 0s 27ms/step
7507 [D loss: 0.3675307631492615] [G loss: 1.9328343868255615]
1/1 [========================

In [5]:
# Save the generator model
generator.save('generator_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


used saved model text to image generator  

In [6]:
!pip install transformers

In [1]:
#Now you can encode text using a pre-trained model:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained model/tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def encode_text(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=20, truncation=True, padding='max_length')
    outputs = model(**inputs)
    text_embedding = outputs.last_hidden_state.mean(dim=1)  # Using mean pooling over sequence length
    return text_embedding.detach().numpy()

/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [1]:
# First, ensure that you have all the required libraries installed:
# !pip install tensorflow transformers torch numpy matplotlib

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

def encode_text(text):
    """
    Encodes the input text into a fixed-size vector using a pre-trained BERT model.
    """
    inputs = tokenizer(text, return_tensors="pt", max_length=20, truncation=True, padding='max_length')
    outputs = bert_model(**inputs)
    text_embedding = outputs.last_hidden_state.mean(dim=1)  # Using mean pooling over sequence length
    return text_embedding.detach().numpy()

def generate_image_from_text(generator_model_path, text):
    """
    Generates an image from the input text using a saved generator model.
    """
    # Load the saved generator model
    generator = load_model(generator_model_path)

    # Encode the input text
    text_embedding = encode_text(text)

    # Create a random noise vector
    noise = np.random.normal(0, 1, (1, 100))

    # Normally the text encoding would need to be incorporated with noise
    # Here we're just using noise for simplicity due to dimensional mismatch; real implementation would need adjustment
    input_vector = noise  # This step needs to be changed for a real model

    # Generate image
    gen_image = generator.predict(input_vector)

    # Rescale images from [-1, 1] to [0, 1] for visualization
    gen_image = 0.5 * gen_image + 0.5

    # Display the image
    plt.imshow(gen_image[0])
    plt.axis('off')
    plt.show()

# Example usage:
generate_image_from_text('generator_model.h5', "A simple text prompt that you want to visualize")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'generator_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)